<a href="https://colab.research.google.com/github/mhabib80/Sample-Projects/blob/master/StrataScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt
%cd '/content/drive/MyDrive/Data Science/Data Sets'

/content/drive/MyDrive/Data Science/Data Sets


# Counting Instances in Text

Find the number of times the words 'bull' and 'bear' occur in the contents.

In [ ]:
gf = pd.read_excel('Bull-Bear Text Search.xlsx')
gf

,filename,contents
0,draft1.txt,The stock exchange predicts a bull market whic...
1,draft2.txt,The stock exchange predicts a bull market whic...
2,final.txt,The stock exchange predicts a bull market whic...


In [ ]:
words = ['bull', 'bear']
pd.DataFrame(zip(words, [gf.apply(lambda l:l['contents'].split().count(word), axis=1).sum() for word in words]), columns=['word', 'nentry'])

,word,nentry
0,bull,3
1,bear,2


# Rank Variance Per Country

Which countries have risen in the rankings based on the number of comments between Dec 2019 vs Jan 2020? 

In [ ]:
fb_comments = pd.read_excel('FB-comments_Count.xlsx')
fb_comments.head()

,user_id,created_at,number_of_comments
0,18,2019-12-29,1
1,25,2019-12-21,1
2,78,2020-01-04,1
3,37,2020-02-01,1
4,41,2019-12-23,1


In [ ]:
fb_users = pd.read_excel('FB-active_users.xlsx')
fb_users.head()

,user_id,name,status,country
0,33,Amanda Leon,open,Australia
1,27,Jessica Farrell,open,Luxembourg
2,18,Wanda Ramirez,open,USA
3,50,Samuel Miller,closed,Brazil
4,82,Jacob York,open,Australia


In [ ]:
#merge the tables
fb = fb_comments[fb_comments.created_at.dt.month.isin([1,12])].merge(fb_users[['user_id', 'country']], left_on='user_id', right_on='user_id')
fb

,user_id,created_at,number_of_comments,country
0,18,2019-12-29,1,USA
1,18,2020-01-31,1,USA
2,18,2019-12-17,1,USA
3,18,2019-12-30,1,USA
4,18,2020-01-02,1,USA
...,...,...,...,...
79,37,2019-12-17,1,Luxembourg
80,31,2020-01-28,1,Denmark
81,31,2020-01-03,1,Denmark
82,22,2019-12-19,1,Canada


In [ ]:
#groupby by country, tally comments in each month
fb_g = fb.groupby(['country', fb.created_at.dt.month]).number_of_comments.sum().unstack().fillna(0)
fb_g

created_at,1,12
country,,
Australia,6.0,6.0
Brazil,3.0,3.0
Canada,0.0,2.0
China,8.0,9.0
Denmark,2.0,0.0
Luxembourg,4.0,3.0
Mali,11.0,8.0
USA,11.0,11.0


In [ ]:
# convert to rankings & compare
fb_r = fb_g.rank(method='dense', ascending=False)
fb_r[fb_r[1] < fb_r[12]]

Index(['Australia', 'Denmark', 'Luxembourg', 'Mali'], dtype='object', name='country')

# New And Existing Users

Calculate the share of new and existing users. Output the month, share of new users, and share of existing users as a ratio.
New users are defined as users who started using services in the current month. Existing users are users who started using services in current month and used services in any previous month.

In [ ]:
fe = pd.read_excel('fact_events.xlsx')
fe['event_month'] = fe['time_id'].dt.month
fe['user_month_start'] = fe.groupby('user_id').event_month.transform(min)
fe.head()

,id,time_id,user_id,customer_id,client_id,event_type,event_id,event_month,user_month_start
0,1,2020-02-28,3668-QPYBK,Sendit,desktop,message sent,3,2,2
1,2,2020-02-28,7892-POOKP,Connectix,mobile,file received,2,2,2
2,3,2020-04-03,9763-GRSKD,Zoomit,desktop,video call received,7,4,4
3,4,2020-04-02,9763-GRSKD,Connectix,desktop,video call received,7,4,4
4,5,2020-02-06,9237-HQITU,Sendit,desktop,video call received,7,2,2


In [ ]:
# fe.groupby('event_month').apply(lambda g : g[g.user_month_start == g.event_month].user_id.nunique()/g.user_id.nunique())

event_month
2    1.000000
3    0.294118
4    0.142857
dtype: float64

In [ ]:
def new_users(g):
  new = g[g.user_month_start == g.event_month].user_id.nunique() / g.user_id.nunique()
  existing = 1 - new
  return pd.Series({'share_new' : new, 'share_existing' : existing})

fe.groupby('event_month').apply(new_users)

,share_new,share_existing
event_month,,
2,1.000000,0.000000
3,0.294118,0.705882
4,0.142857,0.857143


# Average Time Between Steps


Find the average time (in seconds), per product, that needed to progress between steps. You can ignore products that were never used. Output the feature id and the average time.

In [ ]:
fb = pd.read_excel('fb_product_features.xlsx')
fb.head()

,feature_id,user_id,step_reached,timestamp
0,0,0,1,2019-03-11 17:15:00
1,0,0,2,2019-03-11 17:22:00
2,0,0,3,2019-03-11 17:25:00
3,0,0,4,2019-03-11 17:27:00
4,0,1,1,2019-03-11 19:51:00


In [ ]:
fb.groupby('feature_id').apply(lambda g : g.groupby('user_id').timestamp.diff().mean().total_seconds()).dropna()

feature_id
0    186.666666
dtype: float64

# Differences In Movie Ratings

Calculate the average lifetime rating and rating from the second latest film across all actors and all films they had acted in. Output a list of actors, their average lifetime rating, rating from the second latest film, and the difference between the two ratings.

In [ ]:
df = pd.read_csv('Filmography.csv').sort_values('name')
df.head(20)

,name,amg_movie_id,movie_title,role_type,rating,year,id
728,Abigail Breslin,V 578500,Haunter,Normal Acting,4.0,2013,52352
727,Abigail Breslin,V 551200,Ender's Game,Normal Acting,5.0,2013,52351
726,Abigail Breslin,V 570771,August: Osage County,Normal Acting,8.0,2013,52350
1671,Abigail Breslin,V 568482,Zambezia,Voice,NaN,2012,52354
1670,Abigail Breslin,V 583915,Maggie,Normal Acting,NaN,2014,52349
729,Abigail Breslin,V 574290,The Call,Normal Acting,6.0,2013,52353
1195,Adriana Barraza,V 558945,Mariachi Gringo,Normal Acting,NaN,2012,19950
271,Adrien Brody,V 574788,Back to 1942,Normal Acting,3.0,2012,17987
1151,Adrien Brody,V 586040,Third Person,Normal Acting,NaN,2013,17986
1150,Adrien Brody,V 556975,Motor City,Normal Acting,NaN,2014,17985


In [ ]:
df = df.dropna(subset=['rating']).groupby('name').filter(lambda g : g.rating.count() > 1)
# df = df.groupby('name').filter(lambda g : g.rating.notna().sum()>1)
df.groupby('name').rating.mean()

name
Abigail Breslin    5.750000
Alan Arkin         4.500000
Albert Finney      6.500000
Alec Baldwin       5.000000
Amy Adams          6.000000
                     ...   
William H. Macy    6.000000
William Hurt       4.500000
Winona Ryder       5.333333
Woody Allen        3.833333
Woody Harrelson    6.000000
Name: rating, Length: 206, dtype: float64

In [ ]:
def ratings(g):
  d = {}
  d['second_last'] = g[g.id.argsort()==1].rating.values[0]
  d['lifetime'] = g.rating.mean()
  d['variance'] = d['second_last'] - d['lifetime']
  return pd.Series(d)
df.groupby('name').apply(ratings)

,second_last,lifetime,variance
name,,,
Abigail Breslin,5.0,5.750000,-0.750000
Alan Arkin,6.0,4.500000,1.500000
Albert Finney,5.0,6.500000,-1.500000
Alec Baldwin,5.0,5.000000,0.000000
Amy Adams,7.0,6.000000,1.000000
...,...,...,...
William H. Macy,7.0,6.000000,1.000000
William Hurt,6.0,4.500000,1.500000
Winona Ryder,7.0,5.333333,1.666667


# The Cheapest Airline Connection

Create a trips table that lists all the cheapest possible trips that can be done in two or fewer stop

In [ ]:
da = pd.read_excel('Flights.xlsx')
da = da.drop(columns='id')
da

,origin,destination,cost
0,SFO,JFK,500
1,SFO,DFW,200
2,SFO,MCO,400
3,DFW,MCO,100
4,DFW,JFK,200
5,JFK,LHR,1000


In [ ]:
da = da.merge(da, left_on='destination', right_on='origin', how='left').\
      merge(da, left_on='destination_y', right_on='origin', how='left')\
      .drop(columns=['origin_y', 'origin'])
da[['cost_x', 'cost_y', 'cost']]= da[['cost_x', 'cost_y', 'cost']].cumsum(axis=1)

pd.DataFrame(
    np.concatenate([da.iloc[:,[0,i,i+1]] for i in range(1,len(da.columns),2)], axis=0), 
    columns = ['origin', 'destination', 'cost']).\
    groupby(['origin', 'destination']).cost.min().reset_index()


,origin,destination,cost
0,DFW,JFK,200.0
1,DFW,LHR,1200.0
2,DFW,MCO,100.0
3,JFK,LHR,1000.0
4,SFO,DFW,200.0
5,SFO,JFK,400.0
6,SFO,LHR,1400.0
7,SFO,MCO,300.0


# Olympics

Find all Norwegian alpine skiers who participated in 1992 but didn't participate in 1994. Output unique athlete names.

In [ ]:
olympics_athletes_events = pd.read_excel('Olympics.xlsx')
df = olympics_athletes_events[(olympics_athletes_events.team=='Norway') & (olympics_athletes_events.sport == 'Alpine Skiing') ]
df

,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal
289,37710,Ole Kristian Furuseth,M,25.0,182,95.0,Norway,NOR,1992 Winter,1992,Winter,Albertville,Alpine Skiing,Alpine Skiing Men's Combined,NaN
290,61011,Marianne Kjrstad,F,23.0,173,64.0,Norway,NOR,1994 Winter,1994,Winter,Lillehammer,Alpine Skiing,Alpine Skiing Women's Super G,NaN
291,111597,Atle Skrdal,M,27.0,184,83.0,Norway,NOR,1994 Winter,1994,Winter,Lillehammer,Alpine Skiing,Alpine Skiing Men's Downhill,NaN
292,39301,Vibecke Caroline Gedde-Dahl,F,20.0,166,62.0,Norway,NOR,1994 Winter,1994,Winter,Lillehammer,Alpine Skiing,Alpine Skiing Women's Super G,NaN
293,71866,Astrid Ldemel,F,20.0,NaN,NaN,Norway,NOR,1992 Winter,1992,Winter,Albertville,Alpine Skiing,Alpine Skiing Women's Giant Slalom,NaN
294,20,Kjetil Andr Aamodt,M,20.0,176,85.0,Norway,NOR,1992 Winter,1992,Winter,Albertville,Alpine Skiing,Alpine Skiing Men's Giant Slalom,Bronze
295,37710,Ole Kristian Furuseth,M,25.0,182,95.0,Norway,NOR,1992 Winter,1992,Winter,Albertville,Alpine Skiing,Alpine Skiing Men's Super G,NaN
296,120092,Jan Einar Thorsen,M,27.0,181,82.0,Norway,NOR,1994 Winter,1994,Winter,Lillehammer,Alpine Skiing,Alpine Skiing Men's Super G,NaN
297,61011,Marianne Kjrstad,F,23.0,173,64.0,Norway,NOR,1994 Winter,1994,Winter,Lillehammer,Alpine Skiing,Alpine Skiing Women's Slalom,NaN
298,75218,Didrik Marksten,M,20.0,NaN,NaN,Norway,NOR,1992 Winter,1992,Winter,Albertville,Alpine Skiing,Alpine Skiing Men's Slalom,NaN


In [ ]:
df.groupby(['name', 'year']).size().unstack().dropna(subset=[1992])[1994].isna().reset_index().name

0            Astrid Ldemel
1          Didrik Marksten
2       Kjetil Andr Aamodt
3    Ole Kristian Furuseth
Name: name, dtype: object

# Nested Dictionary to Pandas DF

In [ ]:
list = [
        {
        "Student": [{"Exam": 90, "Grade": "a"},
                    {"Exam": 99, "Grade": "b"},
                    {"Exam": 97, "Grade": "c"},
                   ],
        "Name": "Paras Jain"
        },
        {
        "Student": [{"Exam": 89, "Grade": "a"},
                    {"Exam": 80, "Grade": "b"}
                   ],
        "Name": "Chunky Pandey"
        }
       ]
# json.dumps(list)
# ''.join(str(dic) for dic in list)

In [ ]:
df = pd.json_normalize(list, record_path='Student', meta = 'Name')
df

,Exam,Grade,Name
0,90,a,Paras Jain
1,99,b,Paras Jain
2,97,c,Paras Jain
3,89,a,Chunky Pandey
4,80,b,Chunky Pandey


In [ ]:
df.pivot_table(index='Name', columns = 'Grade', values=['Exam'])

Exam            
Grade             a     b     c
Name                           
Chunky Pandey  89.0  80.0   NaN
Paras Jain     90.0  99.0  97.0

# Oldest survivor per passenger class

Find the oldest survivor of each passenger class.
Output the name and the age of the survivor along with the corresponding passenger class.

In [ ]:
titanic = pd.read_csv('Titanic.csv')
titanic.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.050,NaN,S


In [ ]:
#the idxmax returns the indices of the largest age per group which can then be retrieved with the loc function
titanic[titanic.survived==1].groupby('pclass').age.idxmax()

pclass
1    630
2    570
3    483
Name: age, dtype: int64

In [ ]:
titanic.loc[titanic[titanic.survived==1].groupby('pclass').age.idxmax()][['pclass', 'name', 'age']]

,pclass,name,age
630,1,"Barkworth, Mr. Algernon Henry Wilson",80.0
570,2,"Harris, Mr. George",62.0
483,3,"Turkula, Mrs. (Hedwig)",63.0


# Find the number of times each word appears in drafts

In [ ]:
from collections import Counter

google_file_store = pd.read_excel('google_file_store.xlsx')
google_file_store

,filename,contents
0,draft1.txt,The stock exchange predicts a bull market whic...
1,draft2.txt,The stock exchange predicts a bull market whic...
2,final.txt,The stock exchange predicts a bull market whic...


In [ ]:
df = google_file_store[google_file_store.filename.str.contains('draft')]
words = Counter([word for sublist in df.contents.str.split() for word in sublist])
pd.DataFrame(words.items(), columns = ['word', 'nentry'])

,word,nentry
0,The,2
1,stock,2
2,exchange,2
3,predicts,2
4,a,3
5,bull,2
6,market,2
7,which,2
8,would,2
9,make,2


# Most Popular Streets

Find the most popular street for low, moderate, and high-risk categories based on the number of incidents per each category. Output the street name along with the popularity and the corresponding risk category.

In [ ]:
import re
la = pd.read_csv('LA_Inspections.csv')
la.head()

,serial_number,activity_date,facility_name,score,grade,service_code,service_description,employee_id,facility_address,facility_city,facility_id,facility_state,facility_zip,owner_id,owner_name,pe_description,program_element_pe,program_name,program_status,record_id
0,DAQHRSETQ,6/8/2017 0:00,MARGARITAS CAFE,93,A,1,ROUTINE INSPECTION,EE0000006,5026 S CRENSHAW BLVD,LOS ANGELES,FA0023656,CA,90043,OW0004133,"BAZAN, ASCENCION",RESTAURANT (61-150) SEATS HIGH RISK,1638,MARGARITAS CAFE,ACTIVE,PR0011718
1,DA2GQRJOS,3/7/2017 0:00,LAS MOLENDERAS,97,A,1,ROUTINE INSPECTION,EE0000997,2635 WHITTIER BLVD,LOS ANGELES,FA0160416,CA,90023,OW0125379,MARISOL FEREGRINO,RESTAURANT (0-30) SEATS HIGH RISK,1632,LAS MOLENDERAS,INACTIVE,PR0148504
2,DAMQTA46T,3/22/2016 0:00,SANDRA'S TAMALES,93,A,1,ROUTINE INSPECTION,EE0001049,5390 WHITTIER BLVD,LOS ANGELES,FA0171769,CA,90022-4032,OW0178828,SANDRA'S TAMALES INC.,RESTAURANT (0-30) SEATS MODERATE RISK,1631,SANDRA'S TAMALES,ACTIVE,PR0164225
3,DAXMBTIRZ,2/12/2018 0:00,CAFE GRATITUDE,97,A,1,ROUTINE INSPECTION,EE0000828,639 N LARCHMONT BLVD STE #102,LOS ANGELES,FA0058921,CA,90004,OW0005704,CAFE GRATITUDE LARCHMONT LLC,RESTAURANT (61-150) SEATS HIGH RISK,1638,CAFE GRATITUDE,ACTIVE,PR0019854
4,DAK8TBMS0,9/10/2015 0:00,THE WAFFLE,90,A,1,ROUTINE INSPECTION,EE0000709,6255 W SUNSET BLVD STE #105,LOS ANGELES,FA0051830,CA,90028,OW0035796,"THE WAFFLE, LLC",RESTAURANT (61-150) SEATS HIGH RISK,1638,THE WAFFLE,ACTIVE,PR0010922


In [ ]:
#extract the address from the street name
def street_name(address):
  m = re.search('(?<=\s)\w{3,}', address)
  return m.group() if m else 'None'


#extract risk category from pe_description
def risk_cat(desc):
  m = re.search('\w+\sRISK', desc)
  return m.group() if m else 'None'


la = la.assign(street_name = la.facility_address.apply(street_name), risk_category = la.pe_description.apply(risk_cat) )


#return largest values
def funct(g):
  streets = g.groupby('street_name').facility_id.count()
  return streets.nlargest(1, keep='all')

result = la.groupby('risk_category').apply(funct)
# la.groupby('risk_category').apply(lambda g : g.groupby('street_name').facility_id.count().nlargest(1, keep='all'))

result.swaplevel()

In [ ]:
def popular_streets(df):
  #extract the address from the street name
  df['street_name'] =  df.facility_address.apply(lambda address : re.search('(?<=\s)\w{3,}', address).group() )

  #extract risk category from pe_description
  df['risk_category'] = df.pe_description.apply(lambda desc : re.search('\w+\sRISK', desc).group() )

  #return largest values
  table = df.groupby('risk_category').apply(lambda g : g.groupby('street_name').facility_id.count().nlargest(1, keep='all'))

  return table.swaplevel()

popular_streets(la)

# Maximum Number of Employees Reached


Write a query that returns every employee that has ever worked for the company. For each employee, calculate the greatest number of employees that worked for the company during their tenure and the first date that number was reached.

In [2]:
ue = pd.read_csv('Uber_employees.csv', index_col=['id'], usecols=['id','hire_date', 'termination_date'], parse_dates=[1,2])
ue.head()

,hire_date,termination_date
id,,
1,2009-02-03,2016-01-01
2,2009-02-03,NaT
3,2009-02-03,NaT
4,2009-04-15,2014-01-01
5,2009-02-03,NaT


In [ ]:
#fillna for currently employed with the latest month's end
from pandas.tseries.offsets import MonthEnd
range_end = ue.termination_date.max() + MonthEnd(1)
ue = ue.fillna(range_end)
ue

,hire_date,termination_date
id,,
1,2009-02-03,2016-01-01
2,2009-02-03,2019-11-30
3,2009-02-03,2019-11-30
4,2009-04-15,2014-01-01
5,2009-02-03,2019-11-30
...,...,...
96,2019-08-13,2019-11-22
97,2019-11-22,2019-11-30
98,2019-10-09,2019-11-30


In [ ]:
#create an overall timescale for the analysis
total_range = pd.date_range(ue.hire_date.min(), range_end)

#create a hire date_range for each ID
employee_hire_range = ue.apply(lambda x : pd.date_range(x['hire_date'], x['termination_date'], closed='left'), axis=1)

#create a bool matrix comparing each employee's date_range to the overall timescale
hire_matrix = np.array([np.isin(total_range, x) for x in employee_hire_range])

#generate a general daily max of employees reached throughout the total_range
total_range_max = hire_matrix.sum(0)

#create a series for the tital_range dates and total_range max_values
series = pd.Series(index=total_range, data = total_range_max )

In [ ]:
pd.DataFrame([(series[hire_matrix[i]].max(), series[hire_matrix[i]].idxmax()) for i in range(ue.shape[0])], columns=['max_emp', 'mindate'], index=ue.index)

,max_emp,mindate
id,,
1,57,2015-04-10
2,58,2017-02-03
3,58,2017-02-03
4,48,2013-12-23
5,58,2017-02-03
...,...,...
96,39,2019-08-13
97,34,2019-11-22
98,35,2019-10-09


In [ ]:
#note the closed kwarg
pd.date_range(start='1/1/2018', end='1/08/2018', closed='left')

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05', '2018-01-06', '2018-01-07'],
              dtype='datetime64[ns]', freq='D')

# WFM Brand Segmentation based on Customer Activity

Whole Foods would like to segment their store brands into Low, Medium, and High segmentation. The segments are to be based on a customer's average basket size which is defined as (total sales / count of transactions), per customer.

The segment thresholds are as follows:
- If average basket size is more than 30, then Segment is “High”.
- If average basket size is between 20 and 30, then Segment is “Medium”.
- If average basket size is less than 20, then Segment is “Low”.

Summarize the number of unique customers, the total number of transactions, total sales, and average basket size, grouped by store brand and segment for 2017.

In [ ]:
wfm_trans = pd.read_excel('WFM.xlsx', parse_dates=True)
wfm_stores = pd.read_excel('WFM.xlsx', sheet_name='Stores')

In [ ]:
wfm_trans.head()

,customer_id,store_id,transaction_date,transaction_id,product_id,sales
0,1,1,2017-01-06,1,101,13
1,1,1,2017-01-06,1,102,5
2,1,1,2017-01-06,1,103,1
3,2,4,2017-05-06,2,105,20
4,5,4,2017-05-06,5,104,12


In [ ]:
wfm_stores.head()

,store_id,store_brand,location
0,1,Clapham Junction,London
1,2,Camden,London
2,3,Fulham,London
3,4,Kensington,London
4,5,Piccadilly Circus,London


In [ ]:
wfm_trans = wfm_trans[wfm_trans.transaction_date.dt.year==2017]
wfm = pd.merge(wfm_trans , wfm_stores[['store_id','store_brand']])
wfm.head()

,customer_id,store_id,transaction_date,transaction_id,product_id,sales,store_brand
0,1,1,2017-01-06,1,101,13,Clapham Junction
1,1,1,2017-01-06,1,102,5,Clapham Junction
2,1,1,2017-01-06,1,103,1,Clapham Junction
3,1,1,2017-01-01,21,401,10,Clapham Junction
4,15,1,2017-05-06,109,506,70,Clapham Junction


In [ ]:
def store_segmentation(store_brand):

  #groupby customer to generate aggregations per customer
  #total sales, total transactions per customer
  customer = store_brand.groupby('customer_id').agg(cust_sales = ('sales', np.sum), cust_trans = ('transaction_id', 'nunique'))
  #average basket size per customer based on above measures
  customer['cust_avg_basket'] = customer['cust_sales'] / customer['cust_trans']
  #customer segment based on their average basket_size
  customer['segment'] =customer['cust_avg_basket'].apply(lambda x : 'High' if x >=30 else 'Medium' if x >=20 else 'Low')

  #group the above table by segment to generate measures by segment
  segment = customer.groupby('segment').agg(number_customers=('segment', 'size'), total_transactions = ('cust_trans', np.sum), total_sales = ('cust_sales', np.sum))
  segment['segment_avg_basket'] = segment['total_sales'] / segment['total_transactions']
  return segment

wfm.groupby('store_brand').apply(store_segmentation)

number_customers  ...  segment_avg_basket
store_brand                   segment                    ...                    
365 by WFM                    High                   21  ...          337.660000
                              Low                     2  ...           14.666667
Camden                        High                    9  ...          268.823529
                              Low                     1  ...           16.000000
                              Medium                  1  ...           29.500000
Clapham Junction              High                   19  ...          215.195122
                              Low                     1  ...           10.000000
Fulham                        High                   14  ...          277.791667
                              Low                     1  ...           15.000000
Kensington                    High                    5  ...          271.428571
                              Low                     1  ...           12.000000
                              Medium                  1  ...           20.000000
Piccadilly Circus             High                    2  ...          319.000000
                              Medium                  1  ...           20.500000
Richmond                      High                    3  ...          655.000000
Stoke Newington               High                    2  ...          324.333333
Whole Foods Market Daily Shop High                   12  ...          299.076923
                              Low                     1  ...            7.000000

[18 rows x 4 columns]

In [ ]:
wfm.groupby(['store_brand', 'customer_id'])\
.agg(cust_sales = ('sales', np.sum), cust_trans = ('transaction_id', 'nunique'))\
.assign(cust_avg_basket = lambda x : x['cust_sales']/x['cust_trans'])\
.assign(segment = lambda x : x['cust_avg_basket'].transform(lambda x : 'High' if x >=30 else 'Medium' if x >=20 else 'Low'))\
.reset_index()\
.groupby(['store_brand', 'segment'])\
.agg(number_customers = ('customer_id', 'count'),total_transactions = ('cust_trans', 'sum'), total_sales = ('cust_sales', 'sum'))\
.assign(avg_basket = lambda x : x['total_sales'] / x['total_transactions'])

number_customers  ...  avg_basket
store_brand                   segment                    ...            
365 by WFM                    High                   21  ...  337.660000
                              Low                     2  ...   14.666667
Camden                        High                    9  ...  268.823529
                              Low                     1  ...   16.000000
                              Medium                  1  ...   29.500000
Clapham Junction              High                   19  ...  215.195122
                              Low                     1  ...   10.000000
Fulham                        High                   14  ...  277.791667
                              Low                     1  ...   15.000000
Kensington                    High                    5  ...  271.428571
                              Low                     1  ...   12.000000
                              Medium                  1  ...   20.000000
Piccadilly Circus             High                    2  ...  319.000000
                              Medium                  1  ...   20.500000
Richmond                      High                    3  ...  655.000000
Stoke Newington               High                    2  ...  324.333333
Whole Foods Market Daily Shop High                   12  ...  299.076923
                              Low                     1  ...    7.000000

[18 rows x 4 columns]

# New Section